[구글 코랩(Colab)에서 실행하기](https://colab.research.google.com/github/lovedlim/bigdata_analyst_cert/blob/main/part2/ch5/ch5_multi_class_classification.ipynb)

# 1.문제정의
- 도서 참고

# 2. 라이브러리 및 데이터 불러오기

In [22]:
# 라이브러리 및 데이터 불러오기
import pandas as pd

# train = pd.read_csv("train.csv")
# test = pd.read_csv("test.csv")
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch5/train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch5/test.csv")

# 3. 탐색적 데이터 분석(EDA)

In [23]:
train.shape, test.shape

((10000, 21), (10000, 20))

In [24]:
train.head()

,Delay_from_due_date,Num_of_Delayed_Payment,Num_Credit_Inquiries,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly,Monthly_Balance,Credit_Score,Credit_Mix,...,Age,Annual_Income,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Monthly_Inhand_Salary,Changed_Credit_Limit,Outstanding_Debt,Total_EMI_per_month
0,56.0,16.0,11.0,35.598217,120.0,Yes,229.093478,252.385965,1,Bad,...,15.0,36597.56,8.0,10.0,29.0,5.0,3143.796667,22.49,2963.18,122.900223
1,49.0,23.0,12.0,25.553106,120.0,Yes,104.613906,219.105944,1,Bad,...,28.0,32057.30,9.0,8.0,16.0,7.0,2606.441667,1.40,1327.26,164.859426
2,34.0,20.0,6.0,40.039954,174.0,Yes,338.626965,251.265589,1,Bad,...,46.0,75868.80,6.0,10.0,32.0,7.0,6074.400000,3.60,1432.71,297.547446
3,21.0,13.0,8.0,25.711678,143.0,NM,116.816864,259.927960,2,Standard,...,46.0,17092.69,7.0,3.0,19.0,7.0,1695.390833,16.40,1417.06,62.794260
4,19.0,13.0,6.0,39.140463,138.0,Yes,87.262887,626.212330,1,Bad,...,45.0,81471.96,6.0,6.0,25.0,5.0,6763.330000,27.09,2679.69,202.857783


In [25]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Delay_from_due_date       10000 non-null  float64
 1   Num_of_Delayed_Payment    10000 non-null  float64
 2   Num_Credit_Inquiries      10000 non-null  float64
 3   Credit_Utilization_Ratio  10000 non-null  float64
 4   Credit_History_Age        10000 non-null  float64
 5   Payment_of_Min_Amount     10000 non-null  object 
 6   Amount_invested_monthly   10000 non-null  float64
 7   Monthly_Balance           10000 non-null  float64
 8   Credit_Score              10000 non-null  int64  
 9   Credit_Mix                10000 non-null  object 
 10  Payment_Behaviour         10000 non-null  object 
 11  Age                       10000 non-null  float64
 12  Annual_Income             10000 non-null  float64
 13  Num_Bank_Accounts         10000 non-null  float64
 14  Num_Cre

In [26]:
train.describe(include='O')

,Payment_of_Min_Amount,Credit_Mix,Payment_Behaviour
count,10000,10000,10000
unique,3,3,6
top,Yes,Standard,Low_spent_Small_value_payments
freq,5269,4591,3416


In [27]:
test.describe(include='O')

,Payment_of_Min_Amount,Credit_Mix,Payment_Behaviour
count,10000,10000,10000
unique,3,3,6
top,Yes,Standard,Low_spent_Small_value_payments
freq,5167,4590,3498


In [28]:
train.isnull().sum().sum()

0

In [29]:
test.isnull().sum().sum()

0

In [30]:
train['Credit_Score'].value_counts()

Credit_Score
2    5237
1    2978
3    1785
Name: count, dtype: int64

# 4. 데이터 전처리

In [31]:
target = train.pop('Credit_Score')
cols = train.columns[train.dtypes == object]
cols

Index(['Payment_of_Min_Amount', 'Credit_Mix', 'Payment_Behaviour'], dtype='object')

In [32]:
train['Payment_of_Min_Amount'] = train['Payment_of_Min_Amount'].astype('category')
train['Credit_Mix'] = train['Credit_Mix'].astype('category')
train['Payment_Behaviour'] = train['Payment_Behaviour'].astype('category')

test['Payment_of_Min_Amount'] = test['Payment_of_Min_Amount'].astype('category')
test['Credit_Mix'] = test['Credit_Mix'].astype('category')
test['Payment_Behaviour'] = test['Payment_Behaviour'].astype('category')

In [33]:
train[cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   Payment_of_Min_Amount  10000 non-null  category
 1   Credit_Mix             10000 non-null  category
 2   Payment_Behaviour      10000 non-null  category
dtypes: category(3)
memory usage: 29.9 KB


In [34]:
train[cols].head()

,Payment_of_Min_Amount,Credit_Mix,Payment_Behaviour
0,Yes,Bad,High_spent_Medium_value_payments
1,Yes,Bad,High_spent_Small_value_payments
2,Yes,Bad,Low_spent_Large_value_payments
3,NM,Standard,High_spent_Small_value_payments
4,Yes,Bad,High_spent_Medium_value_payments


# 5. 검증 데이터 분할

In [35]:
# 검증 데이터 나누기
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    train,
    target,
    test_size=0.2,
    random_state=0)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((8000, 20), (2000, 20), (8000,), (2000,))

# 6. 머신러닝 학습 및 평가

In [36]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [37]:
# 랜덤포레스트(에러)
# from sklearn.ensemble import RandomForestClassifier
# rf = RandomForestClassifier(random_state=0)
# rf.fit(X_train, y_train)

In [38]:
# LightGBM
import lightgbm as lgb
lgbmc = lgb.LGBMClassifier(random_state=0, verbose=-1)
lgbmc.fit(X_train, y_train)
pred = lgbmc.predict(X_val)
pred

array([2, 2, 2, ..., 3, 2, 2])

In [39]:
accuracy = accuracy_score(y_val, pred)
print('accuracy_score:', accuracy)

f1 = f1_score(y_val, pred, average='macro')
print('f1_score:', f1)

accuracy_score: 0.6985
f1_score: 0.6777379561595467


# 7. 예측 및 결과 파일 생성

In [40]:
pred = lgbmc.predict(test)
pred

array([2, 1, 1, ..., 1, 1, 2])

In [41]:
submit = pd.DataFrame({'pred':pred})
submit.to_csv("result.csv", index=False)

In [42]:
pd.read_csv("result.csv")

,pred
0,2
1,1
2,1
3,2
4,1
...,...
9995,2
9996,2
9997,1
9998,1
